In [2]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats
from ranking_utils import scores_better, rankscore_avgtie, greaters, r_names
import pandas   as pd

num_labels = 100
num_cases = 10
task_name = 'LungCT'


results_dir = Path(Path.cwd(),'L2R_Continous_NPY') ##
if not results_dir.exists():
    ##load and extract data
    !wget https://cloud.imi.uni-luebeck.de/s/cw2m7XHa87piQY2/download -O L2R_Continous_NPY.zip
    !unzip L2R_Continous_NPY.zip
    



discard_events = [] #eg. ['2020_MICCAI', '2022_2023_Continous']
discard_teams = ['2020_MICCAI/initial',
                 '2020_MICCAI/stephanie_haeger_mevis_post',
                 '2020_MICCAI/tony_mok_hkust',
                 '2020_MICCAI/tony_mok_hkust_unsupervised',
                 ] #eg. ['2020_MICCAI/Initial', '2022_2023_Continous/Initial']

results_paths = sorted([x.parent for x in results_dir.glob(f'*/*/{task_name}_tre.npy') if x.is_file()])

###apply filters
results_paths = list(filter(lambda x: x.parent.name not in discard_events and x.parent.name+'/'+x.name not in discard_teams, results_paths))
###move initial to front
initial_index = [i for i, s in enumerate(results_paths) if s.name == 'Initial'][0]
results_paths.insert(0, results_paths.pop(initial_index))
print(*results_paths, sep = "\n")
events, unique_teams = [x.parent.name for x in results_paths],[x.name for x in results_paths]
N = len(unique_teams)

##assert that there are both tre.npy and sd_log_j.npy files
assert all([x.joinpath(f'{task_name}_tre.npy').is_file() for x in results_paths])
assert all([x.joinpath(f'{task_name}_sd_log_j.npy').is_file() for x in results_paths])

/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2021_MICCAI/Initial
/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2020_MICCAI/constance_fourcade_nantes
/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2020_MICCAI/deeds
/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2020_MICCAI/lasse_hansen_uzl
/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2020_MICCAI/nifty
/home/grossbroehmer/storage/staff/christophgrossbroeh/projects/Learn2Reg/local_evaluation/L2REval_all/2023/old_tasks/L2R_Continous_NPY/2020_MICCAI/niklas_gunnarsson_uppsala
/home/grossbroehmer/storage/s

In [3]:
tre_2 = np.zeros((N,num_cases,num_labels))
sdlogj_2 = 5 * np.ones((N,num_cases))
times_2 = 3500 * np.ones((N,num_cases))

print('team','\t\t\t\t','TRE',' sdLogJ', 'times')


for i, r_path in enumerate(results_paths):
    team = r_names(r_path.name)
    if Path ( r_path / f'{task_name}_tre.npy').is_file():
        tre = np.load(r_path / f'{task_name}_tre.npy')
        tre[tre != tre] = 0
        tre_2[i,:,:] = tre
    tre2 = np.nan_to_num(tre, nan=30)
    if Path ( r_path / f'{task_name}_sd_log_j.npy').is_file() and not team == 'Initial':
        sdlogj = np.load(r_path / f'{task_name}_sd_log_j.npy')
        sdlogj_2[i,:] = sdlogj
    else:
        sdlogj_2[i,:] = 5
    if Path ( r_path / f'{task_name}_docker_runtimes.npy').is_file() and not team == 'Initial':
        times = np.load(r_path / f'{task_name}_docker_runtimes.npy')
        times_2[i,:] =  times
    else:
        times_2[i,:] = 3500
    

    if len (team) < 5:
        tab = '\t\t\t\t'
    elif len (team) < 12:
        tab = '\t\t\t'
    elif len (team) < 24:
        tab = '\t\t'
    else:
        tab = '\t'
 
    print(i, team, tab + '%0.3f' % tre_2[i].mean(), '%0.3f' % sdlogj_2[i,:].mean(),
            '%0.3f' % times_2[i,:].mean())

    


team 				 TRE  sdLogJ times
0 Initial 			10.235 5.000 3500.000
1 Epicure 			6.547 0.075 3500.000
2 deeds 			2.265 0.070 43.050
3 PDD-Net 			2.462 0.036 4.223
4 NiftyReg 			2.699 0.100 42.200
5 Gunnarson 			8.999 0.118 30.917
6 ConvexAdam 			1.794 0.061 1.820
7 Driver 			2.658 0.101 2.664
8 Imperial 			1.812 0.108 300.363
9 LaTIM 			1.832 0.051 3500.000
10 LapIRN 			1.976 0.062 10.344
11 MEVIS 			1.676 0.083 95.375
12 PIMed 			2.345 0.041 623.928
13 Lifshitz 			2.259 0.075 2.895
14 Multi-Brain 			6.608 0.082 3500.000
15 Winter 			7.413 0.091 12.030
16 corrField 			1.755 0.050 2.913
17 DingkunLiu 			1.881 0.057 3500.000
18 LapIRNv2 			1.661 0.065 3500.000


In [4]:
##robustify by case
tre2_30 = np.zeros((N,10,30))
for j in range(10):
    idx30 = np.argsort(tre_2[0][j])[-30:]
    for i in range(N):
        tre2_30[i,j] = tre_2[i,j,idx30]


In [5]:
rank_all = np.zeros((N,5))
#TASK2
subset = tre_2.reshape(N,-1)
scores = scores_better(-subset,N)
rank_tre2a = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_tre2a
rank_all[:,1] = rank_tre2a
subset = tre2_30.reshape(N,-1)
scores = scores_better(-subset,N)
rank_tre2b = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_tre2b
subset = sdlogj_2.reshape(N,-1)
scores = scores_better(-subset,N)
rank_jac2 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac2
subset = times_2.reshape(N,-1)
scores = scores_better(-subset,N)
rank_time2 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,4] = rank_time2

In [6]:
rank2 = np.power(np.prod(rank_all,axis=1),.2)
all_rank = np.power(np.prod(rank_all,axis=1),1/5)

idx2 = np.argsort(-all_rank)
print('WITH RUNTIME')
print('#','team','\t\t\t\t','TRE\t\t','TRE30\t\t','sdLogJ\t', ' Runtime\t','rank')


for i in range(N):
    team = r_names(unique_teams[idx2[i]])#.zfill(15).replace("0", " ")
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    else:
        new = ''
    if len (team) < 5:
        tab = '\t\t\t\t'
    elif len (team) < 12:
        tab = '\t\t\t'
    elif len (team) < 24:
        tab = '\t\t'
    else:
        tab = '\t'
    print(i+1,new + team,tab,'%0.4f'%tre_2[idx2[i]].mean(),'\t','%0.4f'%tre2_30[idx2[i]].mean(),'\t','%0.4f'%sdlogj_2[idx2[i]].mean(),' ','%.3f'%times_2[idx2[i]].mean(),'\t','%0.3f'%rank2[idx2[i]])
    


WITH RUNTIME
# team 				 TRE		 TRE30		 sdLogJ	  Runtime	 rank
1 corrField 			 1.7549 	 2.4823 	 0.0496   2.913 	 0.899
2 ConvexAdam 			 1.7943 	 2.7016 	 0.0615   1.820 	 0.839
3 MEVIS 			 1.6764 	 2.3701 	 0.0827   95.375 	 0.794
4 LapIRN 			 1.9763 	 2.9527 	 0.0618   10.344 	 0.719
5 > LapIRNv2 			 1.6611 	 2.2880 	 0.0649   3500.000 	 0.633
6 LaTIM 			 1.8325 	 2.5038 	 0.0507   3500.000 	 0.612
7 PDD-Net 			 2.4618 	 3.8096 	 0.0359   4.223 	 0.576
8 Lifshitz 			 2.2591 	 3.0053 	 0.0746   2.895 	 0.564
9 deeds 			 2.2648 	 2.9112 	 0.0702   43.050 	 0.549
10 Imperial 			 1.8121 	 2.5412 	 0.1076   300.363 	 0.547
11 DingkunLiu 			 1.8815 	 2.4688 	 0.0573   3500.000 	 0.535
12 PIMed 			 2.3445 	 3.2704 	 0.0408   623.928 	 0.519
13 NiftyReg 			 2.6986 	 5.2834 	 0.1000   42.200 	 0.511
14 Driver 			 2.6577 	 3.4999 	 0.1013   2.664 	 0.397
15 Winter 			 7.4129 	 10.1148 	 0.0911   12.030 	 0.367
16 Epicure 			 6.5474 	 10.2902 	 0.0746   3500.000 	 0.264
17 Multi-Brain 			 6.6080

In [7]:
#rank2_withoutRUNTIME = np.power(np.prod(rank_all[:,:3],axis=1),.2)

df = pd.DataFrame(columns=['num','team','TRE','TRE30','sdLogJ','rank'])
all_rank_withoutRUNTIME = np.power(np.prod(rank_all[:,:3],axis=1),1/5)
idx2 = np.argsort(-all_rank_withoutRUNTIME)
print('WITHOUT RUNTIME')
print('#','team','\t\t\t\t','TRE\t\t','TRE30\t\t','sdLogJ\t','rank')
for i in range(N):
    team = r_names(unique_teams[idx2[i]])#.zfill(15).replace("0", " ")
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    else:
        new = ''   
    if len (team) < 5:
        tab = '\t\t\t\t'
    elif len (team) < 12:
        tab = '\t\t\t'
    elif len (team) < 24:
        tab = '\t\t'
    else:
        tab = '\t'
    print(i+1,new + team,tab,'%0.4f'%tre_2[idx2[i]].mean(),'\t','%0.4f'%tre2_30[idx2[i]].mean(),'\t','%0.4f'%sdlogj_2[idx2[i]].mean(),' ','%0.3f'%all_rank_withoutRUNTIME[idx2[i]])
    df.loc[i] = [i+1,team,'%0.4f'%tre_2[idx2[i]].mean(),'%0.4f'%tre2_30[idx2[i]].mean(),'%0.4f'%sdlogj_2[idx2[i]].mean(),'%0.3f'%all_rank_withoutRUNTIME[idx2[i]]]


WITHOUT RUNTIME
# team 				 TRE		 TRE30		 sdLogJ	 rank
1 > LapIRNv2 			 1.6611 	 2.2880 	 0.0649   0.980
2 MEVIS 			 1.6764 	 2.3701 	 0.0827   0.980
3 corrField 			 1.7549 	 2.4823 	 0.0496   0.938
4 ConvexAdam 			 1.7943 	 2.7016 	 0.0615   0.901
5 Imperial 			 1.8121 	 2.5412 	 0.1076   0.847
6 LaTIM 			 1.8325 	 2.5038 	 0.0507   0.847
7 LapIRN 			 1.9763 	 2.9527 	 0.0618   0.818
8 DingkunLiu 			 1.8815 	 2.4688 	 0.0573   0.775
9 NiftyReg 			 2.6986 	 5.2834 	 0.1000   0.754
10 deeds 			 2.2648 	 2.9112 	 0.0702   0.705
11 PIMed 			 2.3445 	 3.2704 	 0.0408   0.626
12 Lifshitz 			 2.2591 	 3.0053 	 0.0746   0.626
13 PDD-Net 			 2.4618 	 3.8096 	 0.0359   0.605
14 Driver 			 2.6577 	 3.4999 	 0.1013   0.533
15 Winter 			 7.4129 	 10.1148 	 0.0911   0.486
16 Multi-Brain 			 6.6080 	 8.7538 	 0.0815   0.409
17 Epicure 			 6.5474 	 10.2902 	 0.0746   0.409
18 Gunnarson 			 8.9988 	 11.2714 	 0.1182   0.320
19 Initial 			 10.2353 	 16.7996 	 5.0000   0.251


In [8]:
print(df)
df.to_csv('LungCT_ranking.csv',index=False)

    num         team      TRE    TRE30  sdLogJ   rank
0     1     LapIRNv2   1.6611   2.2880  0.0649  0.980
1     2        MEVIS   1.6764   2.3701  0.0827  0.980
2     3    corrField   1.7549   2.4823  0.0496  0.938
3     4   ConvexAdam   1.7943   2.7016  0.0615  0.901
4     5     Imperial   1.8121   2.5412  0.1076  0.847
5     6        LaTIM   1.8325   2.5038  0.0507  0.847
6     7       LapIRN   1.9763   2.9527  0.0618  0.818
7     8   DingkunLiu   1.8815   2.4688  0.0573  0.775
8     9     NiftyReg   2.6986   5.2834  0.1000  0.754
9    10        deeds   2.2648   2.9112  0.0702  0.705
10   11        PIMed   2.3445   3.2704  0.0408  0.626
11   12     Lifshitz   2.2591   3.0053  0.0746  0.626
12   13      PDD-Net   2.4618   3.8096  0.0359  0.605
13   14       Driver   2.6577   3.4999  0.1013  0.533
14   15       Winter   7.4129  10.1148  0.0911  0.486
15   16  Multi-Brain   6.6080   8.7538  0.0815  0.409
16   17      Epicure   6.5474  10.2902  0.0746  0.409
17   18    Gunnarson   8.998